In [1]:
import os
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import seaborn as sns
import geopandas as gpd
import pandas as pd
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

import cartopy.crs as ccrs
from cartopy.crs import PlateCarree as plate
import cartopy.io.shapereader as shpreader

import xarray as xr
import atlite

import logging
import warnings

warnings.simplefilter('ignore')
logging.captureWarnings(False)
logging.basicConfig(level=logging.INFO)

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import pandas as pd
pd.set_option('display.max_rows', 1000)
#pd.reset_option('display.max_rows')
#from IPython.display import display
#with pd.option_context('display.max_rows', 100, 'display.max_columns', 10):
#    display(df)
pd.set_option('display.max_colwidth',150)


In [3]:
import contextily as cx

#data of European Offshore wind farms is imported
windfarm_map = gpd.read_file("../data/Windfarm-Shapefiles GB/EMODnet_HA_WindFarms_pg_20220324.shp")

# We just want to analyse HyWind off the coast of Aberdeen
hywind = windfarm_map[windfarm_map['NAME'] == 'Hywind Scotland Pilot Park']
hywind



minx,miny,maxx,maxy = hywind.total_bounds
hywind.centroid
hywind.total_bounds


,OBJECTID,COUNTRY,NAME,N_TURBINES,POWER_MW,STATUS,YEAR,COAST_DIST,AREA_SQKM,NOTES,Shape_Leng,Shape_Area,geometry
271,272,United Kingdom,Hywind Scotland Pilot Park,5,30.0,Production,2017,22488.989404,15.403061,None,0.208214,0.002306,"POLYGON ((-1.30977 57.48393, -1.37822 57.46026, -1.39430 57.47383, -1.39476 57.47445, -1.39474 57.47490, -1.39468 57.47613, -1.39466 57.47682, -1...."


271    POINT (-1.36040 57.48448)
dtype: geometry

array([-1.39476251, 57.4602614 , -1.30977209, 57.50525019])

In [4]:
import pandas as pd
#this data is from the hywind research project
#wind data has already been aggregated
#sadly, wind speeds cover only 30 min-intervals, although in a one second-resolution
n=pd.read_excel(r"..\data\Hywind_Wind_Speed_Data.xlsx")
n.insert(loc=3, column='ATLITE mean wind', value=pd.Series())

n.insert(loc=4, column='Absolute difference', value=pd.Series())
n.insert(loc=5, column='Percentage difference', value=pd.Series())

n
#timestamps =n.loc[:,'Date']

#remove the hour value for ATLITE-cutout
timestamps= n['Date'].dt.date #needs a series #every entry becomes a dateimtime.date object

# according to data description.pdf, dates are in UTC
# ERA5 are also in UTC

type(timestamps)
type(timestamps[0])

,Case no.,Date,Mean wind [m/s],ATLITE mean wind,Absolute difference,Percentage difference,Std. [m/s],Wind direction
0,1,2018-03-26 23:30:00,8.49,NaN,NaN,NaN,0.75,172
1,2,2018-01-14 15:40:00,20.44,NaN,NaN,NaN,2.07,174
2,3,2018-04-14 00:40:00,5.24,NaN,NaN,NaN,0.74,171
3,4,2018-02-13 01:20:00,15.53,NaN,NaN,NaN,1.36,174
4,5,2018-02-24 04:50:00,14.10,NaN,NaN,NaN,1.12,161
5,6,2018-01-09 09:40:00,13.89,NaN,NaN,NaN,1.30,146
6,7,2018-01-06 08:00:00,13.72,NaN,NaN,NaN,1.33,11
7,8,2018-07-29 04:00:00,16.62,NaN,NaN,NaN,1.41,179
8,9,2018-05-02 04:00:00,15.61,NaN,NaN,NaN,1.07,175
9,10,2018-01-24 11:30:00,29.96,NaN,NaN,NaN,3.47,212


pandas.core.series.Series

datetime.date

In [5]:
#data from the hywind catapult project are fragmentary
# only cover singular days
# actually there is no need to create a cutout, one could use the Copernicus API instead
# but since I would have to look up the Copernicus API for location in the docstrings, I will just go with the ATLITE cutout

# in this case: we would just analyse the wind speed at the centroid of the windfarm shapefile

#CDS API code could be adapted from https://github.com/PyPSA/atlite/blob/610edb7e0342d15d6a062a907adb2f64a677aec8/atlite/datasets/era5.py
#https://gist.github.com/matteodefelice/ffd23bd70dbbfc0839f8d5f5e5a9f999 
#in its raw form, u- and v-componentes are called separately, the cutout method is combining both for us

cutout_list=[None] *len(timestamps)
for t in range(len(timestamps)):
    cutout_list[t] = atlite.Cutout(path="wind_data_hywind"+ str(t)+".nc",
                       module="era5",
                       x=slice(minx-0.25, maxx+0.25),
                       y=slice(miny-0.25, maxy+0.25),
                       time=str(timestamps[t]))
    cutout_list[t].prepare()
    
#cutout_list

INFO:atlite.data:Cutout already prepared.


<Cutout "wind_data_hywind0">
 x = -1.50 ⟷ -1.25, dx = 0.25
 y = 57.25 ⟷ 57.75, dy = 0.25
 time = 2018-03-26 ⟷ 2018-03-26, dt = H
 module = era5
 prepared_features = ['height', 'wind', 'influx', 'temperature', 'runoff']

INFO:atlite.data:Cutout already prepared.


<Cutout "wind_data_hywind1">
 x = -1.50 ⟷ -1.25, dx = 0.25
 y = 57.25 ⟷ 57.75, dy = 0.25
 time = 2018-01-14 ⟷ 2018-01-14, dt = H
 module = era5
 prepared_features = ['height', 'wind', 'influx', 'temperature', 'runoff']

INFO:atlite.data:Cutout already prepared.


...

<Cutout "wind_data_hywind8">
 x = -1.50 ⟷ -1.25, dx = 0.25
 y = 57.25 ⟷ 57.75, dy = 0.25
 time = 2018-05-02 ⟷ 2018-05-02, dt = H
 module = era5
 prepared_features = ['height', 'wind', 'influx', 'temperature', 'runoff']

INFO:atlite.data:Cutout already prepared.


<Cutout "wind_data_hywind9">
 x = -1.50 ⟷ -1.25, dx = 0.25
 y = 57.25 ⟷ 57.75, dy = 0.25
 time = 2018-01-24 ⟷ 2018-01-24, dt = H
 module = era5
 prepared_features = ['height', 'wind', 'influx', 'temperature', 'runoff']

INFO:atlite.data:Cutout already prepared.


<Cutout "wind_data_hywind10">
 x = -1.50 ⟷ -1.25, dx = 0.25
 y = 57.25 ⟷ 57.75, dy = 0.25
 time = 2018-01-24 ⟷ 2018-01-24, dt = H
 module = era5
 prepared_features = ['height', 'wind', 'influx', 'temperature', 'runoff']

In [7]:
cutout_list[0].data
#the "data" is necessary to apply selection operations
type(n['Date'][0])

# all pf the dates are pandas timestamps
# xarray works with datetime, need to convert at a point
# we have created several cutouts, so do not mix up the timestamps otherwise: error
# unfortunately, Hywind data starts at points wihtin an hour, ERA5 data always at full hour --> round timestamp down ) to avoid 
# getting to the next day


rounded =n['Date'].dt.round("H")
rounded.iloc[0]="2018-03-26 23:00:00" # the first entry is rounded down so that we avoid the possible crossing of midnight which would require a new cutout
rounded.iloc[0]
#interestingly rounding changes the datatype to dt64, but apparently the dataset would accept any timebased object and convert it itself

import numpy as np
#import datetime
##n.columns
##n.astype({'Date': 'datetime64[ns]'}) #would also be possible
#after all the date format is not decisive, but that the timestamp exists in the xarr --> is full hour

#b= cutout_list[0].data.sel({"x":-1.25,"y":57.5,"time":a[0]}) #direkte Anwahl
type(hywind.centroid.x.values)

for it in range(len(timestamps)):
    c= cutout_list[it].data.sel({"x":hywind.centroid.x.values,"y":hywind.centroid.y.values,"time":rounded[it]},'nearest')
    n['ATLITE mean wind'][it]=c.get('wnd100m').values

    
#both functions calculate for the entire frame, therefore select the result column ATLITE mean wind    
n['Percentage difference'] = n[['Mean wind [m/s]','ATLITE mean wind']].pct_change(axis='columns')['ATLITE mean wind'] 
n['Absolute difference'] = n[['Mean wind [m/s]','ATLITE mean wind']].diff(axis='columns')['ATLITE mean wind']
    
           
n

#d = cutout_list[0].data.sel({"x":-1.25,"y":57.5,"time":a[0]})["wnd100m"].values #zulässige Schreibweise

#b.wnd100m

<xarray.Dataset>
Dimensions:           (x: 2, y: 3, time: 24)
Coordinates:
  * x                 (x) float64 -1.5 -1.25
  * y                 (y) float64 57.25 57.5 57.75
  * time              (time) datetime64[ns] 2018-03-26 ... 2018-03-26T23:00:00
    lon               (x) float64 dask.array<chunksize=(2,), meta=np.ndarray>
    lat               (y) float64 dask.array<chunksize=(3,), meta=np.ndarray>
Data variables: (12/13)
    height            (y, x) float32 dask.array<chunksize=(3, 2), meta=np.ndarray>
    wnd100m           (time, y, x) float32 dask.array<chunksize=(24, 3, 2), meta=np.ndarray>
    wnd_azimuth       (time, y, x) float32 dask.array<chunksize=(24, 3, 2), meta=np.ndarray>
    roughness         (time, y, x) float32 dask.array<chunksize=(24, 3, 2), meta=np.ndarray>
    influx_toa        (time, y, x) float32 dask.array<chunksize=(24, 3, 2), meta=np.ndarray>
    influx_direct     (time, y, x) float32 dask.array<chunksize=(24, 3, 2), meta=np.ndarray>
    ...                ...
    albedo            (time, y, x) float32 dask.array<chunksize=(24, 3, 2), meta=np.ndarray>
    solar_altitude    (time, y, x) float64 dask.array<chunksize=(24, 3, 2), meta=np.ndarray>
    solar_azimuth     (time, y, x) float64 dask.array<chunksize=(24, 3, 2), meta=np.ndarray>
    temperature       (time, y, x) float32 dask.array<chunksize=(24, 3, 2), meta=np.ndarray>
    soil temperature  (time, y, x) float32 dask.array<chunksize=(24, 3, 2), meta=np.ndarray>
    runoff            (time, y, x) float32 dask.array<chunksize=(24, 3, 2), meta=np.ndarray>
Attributes:
    module:             era5
    prepared_features:  ['temperature', 'runoff', 'height', 'wind', 'influx']
    chunksize_time:     100
    Conventions:        CF-1.6
    history:            2023-01-16 15:06:45 GMT by grib_to_netcdf-2.25.1: /op...

pandas._libs.tslibs.timestamps.Timestamp

Timestamp('2018-03-26 23:00:00')

numpy.ndarray

,Case no.,Date,Mean wind [m/s],ATLITE mean wind,Absolute difference,Percentage difference,Std. [m/s],Wind direction
0,1,2018-03-26 23:30:00,8.49,8.962845,0.472845,0.055694,0.75,172
1,2,2018-01-14 15:40:00,20.44,21.638845,1.198845,0.058652,2.07,174
2,3,2018-04-14 00:40:00,5.24,5.508759,0.268759,0.051290,0.74,171
3,4,2018-02-13 01:20:00,15.53,15.603830,0.073830,0.004754,1.36,174
4,5,2018-02-24 04:50:00,14.10,14.165156,0.065156,0.004621,1.12,161
5,6,2018-01-09 09:40:00,13.89,13.422613,-0.467387,-0.033649,1.30,146
6,7,2018-01-06 08:00:00,13.72,14.917101,1.197101,0.087252,1.33,11
7,8,2018-07-29 04:00:00,16.62,16.100603,-0.519397,-0.031251,1.41,179
8,9,2018-05-02 04:00:00,15.61,14.053473,-1.556527,-0.099713,1.07,175
9,10,2018-01-24 11:30:00,29.96,24.788727,-5.171273,-0.172606,3.47,212


**Conclusion**

It appears that the ERA5 reanalysis data is quite precise when compared to historic real-world measurement data, even in consideration of the coarse resolution of the grid cells. This applies appearently to those wind strengths that are the most common, the deviations between calculated and measured data is noteworthy in high wind strength ranges. Nevertheless, wind speeds above 28.5 m/s are considered hurrican-like and by definition rare. Presumably, cases No. 10 and 11 have been selected by the research group of Equinor on purpose to show the behaviour of other components such as the mooring lines under these extreme conditions. It can equally be assumed, that reanalysis data will not depict extreme weather events accurately due to the interpolation between weather stations and in time.
Therefore, the ERA5 data seem to portray sufficiently real wind speeds.

Of course, the validity of this comparison may be limited by the measurement data encompassing only half an hour for each case. Completely different wind speeds might hide in this blind spot, the missing half an hour.
On top of that, since the data are only available in this resolution, the reference value of ERA5 data spans the nearest complete hour, not the very same timescale as the measurements. Thus, not the exact same time spans are compared, but the usage of the mean wind mitigates deviations.
<br>
*Is there maybe literature on the variability of wind over 1 hour-intervalls?*